<h2> 1. Installing Pakages

In [394]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [395]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [396]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


<h2> 2. Importing Pakages

In [429]:
import cv2
import imutils
import pytesseract
import numpy as np

In [449]:
#read the image file
image = cv2.imread('car3.jpg')
# resize or standardize the image
image = imutils.resize(image, width = 500)

In [450]:
#Display the orginal image
cv2.imshow("Original Image", image) #Original Image is the name of window
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2.1 Why turning the image into grayscale?
- It is because it will reduce the dimension, also reduces the complexity of image and there are few algorithms like canny which only works on grayscale images.

In [451]:
#Convert image into gray scale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray Scale Image", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2.2 Bilateral Filtering 
- A bilateral filter is used for smoothening images and reducing noise, while preserving edges. (https://www.geeksforgeeks.org/python-bilateral-filtering/)

In [452]:
#Removing Noise and making it smooth
gray = cv2.bilateralFilter(gray, 11, 17, 17)
cv2.imshow("Smooth Image", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [453]:
#Finding edges of the images
edges = cv2.Canny(gray, 200, 250)
cv2.imshow("Cannyedge", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2.3 What is Hierarchy? (https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html)
- Normally we use the cv.findContours() function to detect objects in an image, right ? Sometimes objects are in different locations. But in some cases, some shapes are inside other shapes. Just like nested figures. In this case, we call outer one as parent and inner one as child. This way, contours in an image has some relationship to each other. And we can specify how one contour is connected to each other, like, is it child of some other contour, or is it a parent etc. Representation of this relationship is called the Hierarchy.

- 1. RETR_LIST It simply retrieves all the contours, but doesn't create any parent-child relationship. This is what we are using here 
2. CHAIN_APPROX_SIMPLE (https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html)

In [454]:
#Find contour based on the image
cnts,new = cv2.findContours(edges.copy() ,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
image1 = image.copy()
cv2.drawContours(image1, cnts, -1, (0,255,0),3) #to draw all contours in image
cv2.imshow("Canny Contour Image", image1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [455]:
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:20]
NumberPlateCount = None
image2= image.copy()
cv2.drawContours(image2, cnts, -1, (0,255,0),3)
cv2.imshow("Top 30 Contour", image2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [456]:
count = 0
name = 1

for i in cnts:
    perimetre = cv2.arcLength(i,True) #perimetre is also called arclength
    approx = cv2.approxPolyDP(i, 0.02*perimetre, True)
    if len(approx)==4:  # 4 means 4 corners of numberplate
        NumberPlateCount = approx
        x,y,w,h = cv2.boundingRect(i)
        crop_image = image[y:y+h, x:x+w]
        cv2.imwrite(str(name)+ '.png', crop_image)
        name+=1
        break
        
#Draw contours in the main image that is identified as numberplate
cv2.drawContours(image,[NumberPlateCount], -1,(0,255,0),3)
cv2.imshow("Final Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [457]:
#cropping the only numberplate 
crop_image_loc = '1.png'
cv2.imshow("Numberplate Image", cv2.imread(crop_image_loc))
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2.4 pytesseract (https://pypi.org/project/pytesseract/)
- Python-tesseract is an optical character recognition (OCR) tool for python. That is, it will recognize and “read” the text embedded in images.
- Python-tesseract is a wrapper for Google’s Tesseract-OCR Engine. It is also useful as a stand-alone invocation script to tesseract, as it can read all image types supported by the Pillow and Leptonica imaging libraries, including jpeg, png, gif, bmp, tiff, and others. Additionally, if used as a script, Python-tesseract will print the recognized text instead of writing it to a file.

In [458]:
#Convetring the numberplate image into txt
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(crop_image_loc, lang='eng')
print("The number of vehical is: ", text)
cv2.waitKey(0)
cv2.destroyAllWindows()

The number of vehical is:  MH 20 EE 7598



## 3. Conclusion

- The Number plates of the car is dectected sucessfully and with the help of pytesseract our final input image is converted into string.